In [ ]:
'''
本补丁包针对性解决十月五日组会将会提出的六个问题中的三个：
2.特征传输；
3.形态学操作；
4.轮廓线上点的排序；
'''

In [ ]:
'''
特征传输

变量表：
vertices—————————————————————加密后点云坐标
faces————————————————————————加密后点云三角面索引
sampled_points———————————————下采样后点云坐标
sampled_indices——————————————下采样后点云在加密后点云中的索引
predictions——————————————————下采样后点云的预测结果（概率形式）
sampled_predictions——————————下采样后点云的预测结果（单个标签的概率）
up_predictions———————————————加密后点云的预测结果（概率形式）

原理：
KNN + 距离反比加权插值
'''
def feature_upsampling(vertices, sampled_indices, sampled_predictions, K=5):
    dic = {i: value for i, value in enumerate(sampled_indices)}
    temp_points = vertices[sampled_indices]
    up_predictions = []
    for i in range(len(vertices)):
        if i in sampled_indices:
            up_predictions.append(sampled_predictions[j for j, v in dic.items() if v == i])
        else:
            P = vertices[i]
            distances = np.linalg.norm(temp_points - P, axis=1)
            idx = np.argsort(distances)[:K]
            dis = np.array(distances[idx])
            weights = 1.0 / dis
            temp_predictions = np.array(sampled_predictions[idx])
            temp_prob = np.dot(weights, temp_predictions)/np.sum(weights)
            up_predictions.append(temp_prob)
    return up_predictions


In [2]:
'''形态学操作'''
import numpy as np
import open3d as o3d
from scipy.spatial import cKDTree

def continuous_morphology_on_point_values(pcd, values, operation='dilation', radius=0.1, k=10):
    """
    对点云的连续值执行形态学操作（腐蚀、膨胀、开运算、闭运算），不改变点云坐标。
    :param pcd: 输入点云 (open3d.geometry.PointCloud)
    :param values: 每个点对应的连续值 (与点云大小相同)
    :param operation: 形态学操作类型 ('dilation', 'erosion', 'opening', 'closing')
    :param radius: 邻域半径，控制搜索邻域的大小
    :param k: k-近邻的 k 值，默认为10 (未使用半径邻域时使用)
    :return: 更新后的连续值数组 (与原点云大小相同)
    """
    # 获取点云的坐标
    points = np.asarray(pcd.points)
    
    # 创建 KD 树用于快速邻域查询
    tree = cKDTree(points)
    
    # 初始化输出值
    new_values = np.copy(values)
    
    if operation == 'dilation':  # 膨胀，取邻域内的最大值
        for i, point in enumerate(points):
            idx = tree.query_ball_point(point, radius)
            new_values[i] = np.max(values[idx])  # 膨胀：邻域中的最大值
    
    elif operation == 'erosion':  # 腐蚀，取邻域内的最小值
        for i, point in enumerate(points):
            idx = tree.query_ball_point(point, radius)
            new_values[i] = np.min(values[idx])  # 腐蚀：邻域中的最小值
    
    elif operation == 'opening':  # 开运算：先腐蚀再膨胀
        eroded_values = np.copy(values)
        # 腐蚀
        for i, point in enumerate(points):
            idx = tree.query_ball_point(point, radius)
            eroded_values[i] = np.min(values[idx])  # 先腐蚀
        # 膨胀
        for i, point in enumerate(points):
            idx = tree.query_ball_point(point, radius)
            new_values[i] = np.max(eroded_values[idx])  # 再膨胀
    
    elif operation == 'closing':  # 闭运算：先膨胀再腐蚀
        dilated_values = np.copy(values)
        # 膨胀
        for i, point in enumerate(points):
            idx = tree.query_ball_point(point, radius)
            dilated_values[i] = np.max(values[idx])  # 先膨胀
        # 腐蚀
        for i, point in enumerate(points):
            idx = tree.query_ball_point(point, radius)
            new_values[i] = np.min(dilated_values[idx])  # 再腐蚀
    
    return new_values

# 示例用法

# 创建示例点云
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.random.rand(100, 3))  # 100 个随机点

# 生成每个点的连续值
values = np.random.rand(100)  # 随机生成 0-1 范围内的连续值

# 执行形态学操作 (例如膨胀)
new_values = continuous_morphology_on_point_values(pcd, values, operation='dilation', radius=0.1)

# 输出原始值和更新后的值
print("原始连续值: ", values)
print("膨胀后的连续值: ", new_values)


原始连续值:  [0.64702848 0.41364331 0.58812899 0.64062785 0.2119826  0.08266967
 0.53540976 0.35351991 0.12659988 0.27923153 0.7349676  0.93958152
 0.60344079 0.93561099 0.53558947 0.24691247 0.02172969 0.24566488
 0.70818915 0.31484305 0.88064235 0.6315853  0.88620381 0.04468645
 0.41971056 0.82128935 0.25711483 0.5648368  0.28368986 0.3375532
 0.76581844 0.41134282 0.12224418 0.97663423 0.82439134 0.87835632
 0.35372692 0.24441686 0.47609569 0.87997985 0.82395616 0.9837165
 0.19886446 0.57539428 0.91288964 0.5174324  0.62401842 0.38569965
 0.29635877 0.5480986  0.53812863 0.03966005 0.27238921 0.28636945
 0.358035   0.17703109 0.47167041 0.20301145 0.77478113 0.80319477
 0.73609812 0.55013582 0.25031075 0.81225612 0.12225703 0.80535803
 0.2300456  0.05987181 0.34130796 0.97939599 0.56193221 0.56992042
 0.18298461 0.83435579 0.43568818 0.19093589 0.49426713 0.89383025
 0.26967191 0.85607311 0.14544841 0.00492388 0.04537847 0.07548048
 0.60799956 0.08369367 0.30577206 0.46067569 0.28695888 

In [ ]:
'''轮廓线上点的排序'''

'''
['Ya Gen', 'Qie Duan', 'Jin Zhong', 'Yuan Zhong', 'She Zhou Mian', 'Chun Mian', 'Jian Tai', 'She Mian Wo']
(1,2)
(2,3)(2,4)(2,6)(2,8)
(3,5)(3,6)(3,7)(3,8)
(4,5)(4,6)(4,7)(4,8)
(5,7)(5,8)
(6,7)
'''

for i in range(7):
    for j in range(i+1,8):
        boundary_edges = all_boundary_edges[(i+1,j+1)]
        boundary_midpoints = calculate_midpoints(boundary_edges, np.array(vertices))

        if boundary_midpoints.size >0:
            if i==0 and j==6:
                '''牙根-肩台'''
                reference_point = np.array([0, 0, 0])  # 参考点
                reference_direction = np.array([0, 0, 1])  # 参考方向
            elif i==1 and j==2:
                '''切端-近中'''
                reference_point = np.array([0, 0, 6])  # 参考点
                reference_direction = np.array([0, 0, 1])  # 参考方向
            elif i==1 and j==3:
                '''切端-远中'''
                reference_point = np.array([0, 0, 6])  # 参考点
                reference_direction = np.array([0, 0, 1])  # 参考方向
            elif i==1 and j==5:
                '''切端-唇面'''
                reference_point = np.array([0, 6, 6])  # 参考点
                reference_direction = np.array([1, 0, 0])  # 参考方向
            elif i==1 and j==7:
                '''切端-舌面窝'''
                reference_point = np.array([0, 6, -6])  # 参考点
                reference_direction = np.array([1, 0, 0])  # 参考方向
            elif i==2 and j==4:
                '''近中-舌轴面'''
                reference_point = np.array([0, 0, 0])  # 参考点
                reference_direction = np.array([0, 1, 0])  # 参考方向
            elif i==2 and j==5:
                '''近中-唇面'''
                reference_point = np.array([0, -2, 0])  # 参考点
                reference_direction = np.array([0, 0, -1])  # 参考方向
            elif i==2 and j==6:
                '''近中-肩台'''
                reference_point = np.array([4, 6, 0])  # 参考点
                reference_direction = np.array([0, 0, 1])  # 参考方向
            elif i==2 and j==7:
                '''近中-舌面窝'''
                reference_point = np.array([4, 4, -4])  # 参考点
                reference_direction = np.array([0, 1, 0])  # 参考方向
            elif i==3 and j==4:
                '''远中-舌轴面'''
                reference_point = np.array([0, 0, 0])  # 参考点
                reference_direction = np.array([0, 1, 0])  # 参考方向
            elif i==3 and j==5:
                '''远中-唇面'''
                reference_point = np.array([0, -2, 0])  # 参考点
                reference_direction = np.array([0, 0, -1])  # 参考方向
            elif i==3 and j==6:
                '''远中-肩台'''
                reference_point = np.array([-4, 6, 0])  # 参考点
                reference_direction = np.array([0, 0, 1])  # 参考方向
            elif i==3 and j==7:
                '''远中-舌面窝'''
                reference_point = np.array([-4, 4, -4])  # 参考点
                reference_direction = np.array([0, 1, 0])  # 参考方向
            elif i==4 and j==6:
                '''舌轴面-肩台'''
                reference_point = np.array([0, 0, 0])  # 参考点
                reference_direction = np.array([0, 0, 1])  # 参考方向
            elif i==4 and j==7:
                '''舌轴面-舌面窝'''
                reference_point = np.array([0, 0, 0])  # 参考点
                reference_direction = np.array([0, 0, 1])  # 参考方向
            elif i==5 and j==6:
                '''唇面-肩台'''
                reference_point = np.array([0, 0, 0])  # 参考点
                reference_direction = np.array([0, 0, 1])  # 参考方向

            # 确保参考方向是单位向量
            reference_direction = reference_direction / np.linalg.norm(reference_direction)
            
            # 计算每个点与参考点的相对向量
            relative_points = boundary_midpoints - reference_point  # 计算相对坐标
            
            # 计算与参考方向的夹角
            angles = np.arccos(np.clip(np.dot(relative_points, reference_direction), -1.0, 1.0))
            
            # 获取排序的索引
            sorted_indices = np.argsort(angles)
            
            # 返回排序后的点集
            sorted_points = boundary_midpoints[sorted_indices]
